In [10]:
import os
import json
import cobrakbase
import cobra
from memote.suite.cli.reports import report
import memote.suite.api as api
from memote.suite.reporting import ReportConfiguration
from urllib.request import urlopen
import pandas as pd
import memote.utils as utils

kbase = cobrakbase.KBaseAPI(token='BAWCY2DLIT7VDVXDUSQCSERVAEZFJR2I', dev=False)

In [10]:
cpd_ref_file  = '/Users/fliu/workspace/kbase/kb_memote/data/Unique_ModelSEED_Compound_Aliases.txt'
rxn_ref_file  = '/Users/fliu/workspace/kbase/kb_memote/data/Unique_ModelSEED_Reaction_Aliases.txt'
rxn_ec_file   = '/Users/fliu/workspace/kbase/kb_memote/data/Unique_ModelSEED_Reaction_ECs.txt'
cpd_stru_file = '/Users/fliu/workspace/kbase/kb_memote/data/ModelSEED_Structures.txt'

cpd_df    = pd.read_csv(cpd_ref_file,  sep='\t')
rxn_df    = pd.read_csv(rxn_ref_file,  sep='\t')
rxn_ec_df = pd.read_csv(rxn_ec_file,   sep='\t')
stru_df   = pd.read_csv(cpd_stru_file, sep='\t')
        
structures   = cobrakbase.read_modelseed_compound_structures(stru_df)
rxn_aliases  = cobrakbase.read_modelseed_reaction_aliases2(rxn_df)
cpd_aliases  = cobrakbase.read_modelseed_compound_aliases2(cpd_df)
gene_aliases = cobrakbase.read_genome_aliases(genome)

In [ ]:
genome1 = kbase.get_object('', '')

In [2]:
genome1 = kbase.get_object('', 'filipeliu:narrative_1542045014574')

In [11]:
genome1 = kbase.get_object('Escherichia_coli_str._K-12_substr._MG1655', 'filipeliu:narrative_1542045014574')


In [17]:

def get_old_alias(alias, feature, gene_aliases):
    if alias.startswith('EcoGene:'):
        gene_aliases[feature['id']]['ecogene'] = alias.split(':')[1]
    elif alias.startswith('UniProtKB/Swiss-Prot:'):
        gene_aliases[feature['id']]['uniprot'] = alias.split(':')[1]
    elif alias.startswith('NP_'):
        gene_aliases[feature['id']]['ncbiprotein'] = alias
    elif alias.startswith('ASAP:'):
        gene_aliases[feature['id']]['asap'] = alias.split(':')[1]
    elif alias.startswith('GI:'):
        gene_aliases[feature['id']]['ncbigi'] = alias
    elif alias.startswith('GeneID:'):
        gene_aliases[feature['id']]['ncbigene'] = alias.split(':')[1]
    else:
        1
        
def read_genome_aliases(genome):
    gene_aliases = {}
    for feature in genome['features']:
        gene_aliases[feature['id']] = {}
        if 'aliases' in feature and type(feature['aliases']) == list:
            for alias in feature['aliases']:
                if type(alias) == list:
                    for a in alias:
                        #risk parsing the first element
                        get_old_alias(a, feature, gene_aliases)
                else:
                    get_old_alias(alias, feature, gene_aliases)
                    #print('discard', alias)
        #print(feature['aliases' in ])
    return gene_aliases

In [35]:
FEATURE_XREF_MAPPING = {
    'GeneID' : 'ncbigene',
    'EcoGene' : 'ecogene',
#    'GeneID' : 'ncbigene',
}
def get_namespace(db_xref):
    if db_xref in FEATURE_XREF_MAPPING:
        return FEATURE_XREF_MAPPING[db_xref]
    return None

In [41]:
gene_aliases = {}
for feature in genome['features']:
    #print(feature)
    gene_aliases[feature['id']] = {}
    if 'aliases' in feature and type(feature['aliases']) == list:
        for alias in feature['aliases']:
            if type(alias) == list:
                for a in alias:
                    #risk parsing the first element
                    get_old_alias(a, feature, gene_aliases)
            else:
                get_old_alias(alias, feature, gene_aliases)
    if 'db_xrefs' in feature:
        for db_xref in feature['db_xrefs']:
            #print(db_xref)
            db = get_namespace(db_xref[0])
            if not db == None:
                gene_aliases[feature['id']][db] = db_xref[1]
            else:
                print(db_xref)
#gene_aliases

In [11]:
gene_aliases['b0002']

{'ncbiprotein': 'NP_414543.1'}

In [8]:
kbase = cobrakbase.KBaseAPI(token='OYSFZMHFURU3TBB5AEOT3EGAYV6TGICS', dev=False)
kmodel = kbase.get_object("ecoli_model", 'jplfaria:narrative_1543955108091')
media, genome = get_media_and_genome(kmodel)

Pulling media from gapfilling... [{'gapfill_id': 'ecoli_model.gf.0', 'id': 'gf.0', 'integrated': 1, 'integrated_solution': '0', 'media_ref': '38412/16/1'}]
Loading genome... 38412/14/1


In [ ]:
kbase = cobrakbase.KBaseAPI(token='OYSFZMHFURU3TBB5AEOT3EGAYV6TGICS', dev=True)
kmodel = kbase.get_object("ecoli.model", '')

In [31]:
kbase = cobrakbase.KBaseAPI(token='OYSFZMHFURU3TBB5AEOT3EGAYV6TGICS', dev=False)
kmodel = kbase.get_object("Re-imported_Model", 'jplfaria:narrative_1544216708501')
media_constraints, genome = get_media_and_genome(kmodel)
model = cobrakbase.convert_kmodel(kmodel, media_constraints)

Loading genome... 598/206/1
Setup Drains. EX: 169 SK: 0


NameError: name 'warn' is not defined

In [4]:
def get_media_and_genome(kmodel):
    media_constraints = {}
    if 'gapfillings' in kmodel and len(kmodel['gapfillings']) > 0:
        print("Pulling media from gapfilling...", kmodel['gapfillings'])
        ref = kmodel['gapfillings'][0]['media_ref']
        ref_data = kbase.get_object_info_from_ref(ref)['infos'][0]
        o_id = ref_data[1]
        o_ws = ref_data[7]
        o_data = kbase.get_object(o_id, o_ws)
        media_constraints = cobrakbase.convert_media(o_data)
    print("Loading genome...", kmodel['genome_ref'])
    ref_data = kbase.get_object_info_from_ref(kmodel['genome_ref'])['infos'][0]
    o_id = ref_data[1]
    o_ws = ref_data[7]
    genome = kbase.get_object(o_id, o_ws)
    
    return media_constraints, genome

In [33]:
for o in kbase.list_objects('jplfaria:narrative_1543955108091'):
    print(o)

[1, 'Narrative.1543955108091', 'KBaseNarrative.Narrative-4.0', '2018-12-10T20:56:03+0000', 84, 'filipeliu', 38412, 'jplfaria:narrative_1543955108091', 'b8a6f2466bd2a30affd4ef2e2a16bd55', 114023, None]
[3, 'Escherichia_coli_str._K-12_substr._MG1655', 'KBaseGenomes.Genome-9.0', '2018-12-04T21:41:08+0000', 1, 'jplfaria', 38412, 'jplfaria:narrative_1543955108091', '50f10e3f870236d2d3d98b0782948e59', 16535414, None]
[4, 'Pseudomonas_aeruginosa_PAO1', 'KBaseGenomes.Genome-9.0', '2018-12-04T21:41:48+0000', 1, 'jplfaria', 38412, 'jplfaria:narrative_1543955108091', '87adbc5e2cdd93b449b6c2adad68519e', 20596129, None]
[5, 'Bacillus_subtilis_subsp._subtilis_str._168', 'KBaseGenomes.Genome-9.0', '2018-12-04T21:52:21+0000', 1, 'jplfaria', 38412, 'jplfaria:narrative_1543955108091', '19299df2702fe8d7de33d8266d3622f4', 14950446, None]
[6, 'Salmonella_enterica_subsp._enterica_serovar_Typhimurium_str._LT2', 'KBaseGenomes.Genome-9.0', '2018-12-04T21:52:45+0000', 1, 'jplfaria', 38412, 'jplfaria:narrative_1

In [47]:
models = [('shewanella', 'jplfaria:narrative_1543955108091'), ('salmonella_model', 'jplfaria:narrative_1543955108091'), 
          ('pseudomonas_model', 'jplfaria:narrative_1543955108091'), ('bacillus_model', 'jplfaria:narrative_1543955108091'), ('ecoli_model', 'jplfaria:narrative_1543955108091')]
#models = [('shewanella', 'jplfaria:narrative_1543955108091')]
kbase = cobrakbase.KBaseAPI(token='OYSFZMHFURU3TBB5AEOT3EGAYV6TGICS', dev=False)
for model_id, ws in models:
    kmodel = kbase.get_object(model_id, ws)
    media_constraints, genome = get_media_and_genome(kmodel)
    model = cobrakbase.convert_kmodel(kmodel, media_constraints)
    model = fix_model(model)
    model.summary()
    cobra.io.write_sbml_model(model, model_id + '.xml')

Pulling media from gapfilling... [{'gapfill_id': 'shewanella.gf.0', 'id': 'gf.0', 'integrated': 1, 'integrated_solution': '0', 'media_ref': '38412/16/1'}]
Loading genome... 38412/10/1
Add Sink cpd15302_c0
Add Sink cpd02701_c0
Add Sink cpd11416_c0
Setup Drains. EX: 71 SK: 3
IN FLUXES             OUT FLUXES             OBJECTIVES
--------------------  ---------------------  -------------------
cpd00013_e0  6.53     cpd00001_e0  15        bio1_biomass  0.762
cpd00027_e0  5        cpd00067_e0   8.5
cpd00009_e0  0.618    cpd00007_e0   3.59
cpd00048_e0  0.169    cpd15378_e0   0.00236
cpd10516_e0  0.00943
cpd00030_e0  0.00236
cpd00034_e0  0.00236
cpd00058_e0  0.00236
cpd00063_e0  0.00236
cpd00099_e0  0.00236
cpd00149_e0  0.00236
cpd00205_e0  0.00236
cpd00254_e0  0.00236
Pulling media from gapfilling... [{'gapfill_id': 'salmonella_model.gf.0', 'id': 'gf.0', 'integrated': 1, 'integrated_solution': '0', 'media_ref': '38412/16/1'}]
Loading genome... 38412/11/1
Add Sink cpd15302_c0
Add Sink cpd027

In [43]:
def fix_model(model):
    for m in model.metabolites:
        seed_id = None
        if 'seed.compound' in m.annotation:
            seed_id = m.annotation['seed.compound']
        if seed_id in structures:
            m.annotation.update(structures[seed_id])
        if seed_id in cpd_aliases:
            m.annotation.update(cpd_aliases[seed_id])

    for r in model.reactions:
        seed_id = None
        if 'seed.reaction' in r.annotation:
            seed_id = r.annotation['seed.reaction']
        if seed_id in rxn_aliases:
            r.annotation.update(rxn_aliases[seed_id])

    for g in model.genes:
        if g.id in gene_aliases:
            g.annotation.update(gene_aliases[g.id])

    kbase_sinks = ['rxn13783_c0', 'rxn13784_c0', 'rxn13782_c0']
    for r in model.reactions:
        if cobrakbase.is_translocation(r):
            if cobrakbase.is_transport(r):
                r.annotation['sbo'] = 'SBO:0000655'
            else:
                r.annotation['sbo'] = 'SBO:0000185'
        #r.annotation['ec-code'] = '1.1.1.1'
        #r.annotation['metanetx.reaction'] = 'MNXR103371'
        if r.id in kbase_sinks:
            r.annotation['sbo'] = 'SBO:0000632'
        if r.id.startswith('DM_'):
            r.annotation['sbo'] = 'SBO:0000628'
    return model

In [17]:
kbase = cobrakbase.KBaseAPI(token='OYSFZMHFURU3TBB5AEOT3EGAYV6TGICS', dev=False)

In [18]:
kmodel = kbase.get_object("ecoli_model", 'jplfaria:narrative_1543955108091')

In [19]:
kmodel['genome_ref']

'38412/14/1'

In [11]:
#for r in kmodel['modelreactions']:
#    print(r['id'], r['maxforflux'], r['maxrevflux'])

In [6]:
media_constraints = None

In [3]:
media_constraints = None
with open("glucose_media.json", 'r') as f:
    media_constraints = json.loads(f.read())

In [3]:
kmodel = None
with open("test_model.json", 'r') as f:
    kmodel = json.loads(f.read())

In [12]:
ref_data = kbase.get_object_info_from_ref(kmodel['genome_ref'])['infos'][0]
o_id = ref_data[1]
o_ws = ref_data[7]
genome = kbase.get_object(o_id, o_ws)

In [20]:
if 'gapfillings' in kmodel and len(kmodel['gapfillings']) > 0:
    ref = kmodel['gapfillings'][0]['media_ref']
    ref_data = kbase.get_object_info_from_ref(ref)['infos'][0]
    o_id = ref_data[1]
    o_ws = ref_data[7]
    o_data = kbase.get_object(o_id, o_ws)
    media_constraints = cobrakbase.convert_media(o_data)

In [21]:
model = cobrakbase.convert_kmodel(kmodel, media_constraints)
html = ""
model.summary()

Add Sink cpd15302_c0
Add Sink cpd02701_c0
Add Sink cpd11416_c0
Setup Drains. EX: 172 SK: 3
IN FLUXES             OUT FLUXES             OBJECTIVES
--------------------  ---------------------  -------------------
cpd00013_e0  6.53     cpd00001_e0  16.7      bio1_biomass  0.762
cpd00027_e0  5        cpd00067_e0   5.18
cpd00009_e0  0.618    cpd00007_e0   2.76
cpd00048_e0  0.169    cpd15378_e0   0.00236
cpd10516_e0  0.00943
cpd00030_e0  0.00236
cpd00034_e0  0.00236
cpd00058_e0  0.00236
cpd00063_e0  0.00236
cpd00099_e0  0.00236
cpd00149_e0  0.00236
cpd00205_e0  0.00236
cpd00254_e0  0.00236


In [22]:
cobra.io.write_sbml_model(model, 'export_mode.xml')

In [8]:
#use urls for now but later get from local
data = urlopen('https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/dev/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt')
cpd_df = pd.read_csv(data, sep='\t')

data = urlopen('https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/dev/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt')
rxn_df = pd.read_csv(data, sep='\t')

data = urlopen('https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/dev/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt')
rxn_ec_df = pd.read_csv(data, sep='\t')

data = urlopen('https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/dev/Biochemistry/Structures/ModelSEED_Structures.txt')
stru_df = pd.read_csv(data, sep='\t')

In [9]:
structures  = cobrakbase.read_modelseed_compound_structures(stru_df)
rxn_aliases = cobrakbase.read_modelseed_reaction_aliases2(rxn_df)
cpd_aliases = cobrakbase.read_modelseed_compound_aliases2(cpd_df)
gene_aliases = cobrakbase.read_genome_aliases(genome)

In [12]:
print(cpd_aliases['cpd00022'])
print(gene_aliases['b0002'])

{'biocyc': 'ACETYL-COA', 'kegg.compound': 'C00024', 'bigg.metabolite': 'accoa'}
{'ncbiprotein': 'NP_414543.1'}


In [35]:
#gene_aliases

In [14]:
for m in model.metabolites:
    seed_id = None
    if 'seed.compound' in m.annotation:
        seed_id = m.annotation['seed.compound']
    if seed_id in structures:
        m.annotation.update(structures[seed_id])
    if seed_id in cpd_aliases:
        m.annotation.update(cpd_aliases[seed_id])

for r in model.reactions:
    seed_id = None
    if 'seed.reaction' in r.annotation:
        seed_id = r.annotation['seed.reaction']
    if seed_id in rxn_aliases:
        r.annotation.update(rxn_aliases[seed_id])
        
for g in model.genes:
    if g.id in gene_aliases:
        g.annotation.update(gene_aliases[g.id])
        
for r in model.reactions:
    if cobrakbase.is_translocation(r):
        if cobrakbase.is_transport(r):
            r.annotation['sbo'] = 'SBO:0000655'
        else:
            r.annotation['sbo'] = 'SBO:0000185'
            
kbase_sinks = ['rxn13783_c0', 'rxn13784_c0', 'rxn13782_c0']
    
for r in model.reactions:
    #r.annotation['ec-code'] = '1.1.1.1'
    #r.annotation['metanetx.reaction'] = 'MNXR103371'
    if r.id in kbase_sinks:
        r.annotation['sbo'] = 'SBO:0000632'
    if r.id.startswith('DM_'):
        r.annotation['sbo'] = 'SBO:0000628'

In [15]:
for m in model.metabolites:
    print(m.id, m.annotation)
    break

cpd00443_c0 {'sbo': 'SBO:0000247', 'seed.compound': 'cpd00443', 'inchikey': 'ALYNCZNDIQEVRV-UHFFFAOYSA-M', 'bigg.metabolite': '4abz', 'kegg.compound': 'C00568', 'biocyc': 'P-AMINO-BENZOATE'}


In [9]:
results['tests'].keys()

dict_keys(['test_metabolite_annotation_presence', 'test_reaction_annotation_presence', 'test_gene_product_annotation_presence', 'test_metabolite_annotation_overview', 'test_reaction_annotation_overview', 'test_gene_product_annotation_overview', 'test_metabolite_annotation_wrong_ids', 'test_reaction_annotation_wrong_ids', 'test_gene_product_annotation_wrong_ids', 'test_metabolite_id_namespace_consistency', 'test_reaction_id_namespace_consistency', 'test_model_id_presence', 'test_genes_presence', 'test_reactions_presence', 'test_metabolites_presence', 'test_metabolites_formula_presence', 'test_metabolites_charge_presence', 'test_gene_protein_reaction_rule_presence', 'test_ngam_presence', 'test_metabolic_coverage', 'test_compartments_presence', 'test_protein_complex_presence', 'test_find_pure_metabolic_reactions', 'test_find_constrained_pure_metabolic_reactions', 'test_find_transport_reactions', 'test_find_constrained_transport_reactions', 'test_transport_reaction_gpr_presence', 'test_fin

In [17]:
config = ReportConfiguration.load()

#results.keys()
#api.snapshot_report(results)
#report = api.SnapshotReport()
#report = api.snapshot_report(results, config)
html = api.snapshot_report(results, config)
with open("report_latest.html", "w") as text_file:
    print(html, file=text_file)

In [41]:
from memote.suite.collect import ResultCollectionPlugin
plugin = ResultCollectionPlugin(model, exclusive=None, skip=None, experimental_config=None)

In [97]:
def is_transport(r):
    lhs = set()
    rhs = set()
    for m in r.reactants:
        if 'seed.compound' in m.annotation:
            lhs.add(m.annotation['seed.compound'])
        else:
            return False
    for m in r.products:
        if 'seed.compound' in m.annotation:
            rhs.add(m.annotation['seed.compound'])
        else:
            return False
    
    if not len(lhs & rhs) == 0:
        return True
    
    return False

def is_translocation(r):
    cmps = set()
    
    for m in r.reactants:
        cmps.add(m.compartment)
    for m in r.products:
        cmps.add(m.compartment)
    
    return len(cmps) > 1


        
a = ['rxn13783_c0', 'rxn13784_c0', 'rxn13782_c0']

counter = 0
for o in model.metabolites:
    r.annotation['metanetx.chemical'] = 'MNXM' + str(counter)
    counter += 1
    
for r in model.reactions:
    r.annotation['ec-code'] = '1.1.1.1'
    r.annotation['metanetx.reaction'] = 'MNXR103371'
    if r.id in a:
        r.annotation['sbo'] = 'SBO:0000632'
    if r.id.startswith('DM_'):
        r.annotation['sbo'] = 'SBO:0000632'

rxn13783_c0
rxn13784_c0
rxn13782_c0


In [89]:
model.reactions.get_by_id('rxn08257_c0').annotation

{'sbo': 'SBO:0000176', 'seed.reaction': 'rxn08257', 'bigg.reaction': 'CRNCBCT'}

In [16]:
a, results = api.test_model(model, results=True, skip=['test_thermodynamics', 'test_consistency', 'test_growth', 'test_matrix'])

================================================================================== test session starts ===================================================================================
platform darwin -- Python 3.6.5, pytest-3.9.1, py-1.7.0, pluggy-0.8.0
rootdir: /anaconda3/lib/python3.6/site-packages/memote/suite/tests, inifile:
plugins: rerunfailures-4.2, remotedata-0.2.1, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 140 items

../../../../../../../anaconda3/lib/python3.6/site-packages/memote/suite/tests/test_annotation.py ...FF.FFFFFFFFFFFFFFFFFFFFFFFFFF...FFFFFFFFFFFFF.FFFFFFFFFF..                     [ 43%]
../../../../../../../anaconda3/lib/python3.6/site-packages/memote/suite/tests/test_basic.py ......FF.F........F..                                                                  [ 58%]
../../../../../../../anaconda3/lib/python3.6/site-packages/memote/suite/tests/test_biomass.py .F..FF...F                                                                         

In [8]:
a, results = api.test_model(model, results=True)

================================================================================== test session starts ===================================================================================
platform darwin -- Python 3.6.5, pytest-3.9.1, py-1.7.0, pluggy-0.8.0
rootdir: /anaconda3/lib/python3.6/site-packages/memote/suite/tests, inifile:
plugins: rerunfailures-4.2, remotedata-0.2.1, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 140 items

../../../../../../../anaconda3/lib/python3.6/site-packages/memote/suite/tests/test_annotation.py ...FF.FFFFFFFFFFFFFFFFFFFFFFFFFF...FFFF.FFFFFFFF.FFFFFFFFFF..                     [ 43%]
../../../../../../../anaconda3/lib/python3.6/site-packages/memote/suite/tests/test_basic.py ......FF.F........F..                                                                  [ 58%]
../../../../../../../anaconda3/lib/python3.6/site-packages/memote/suite/tests/test_biomass.py .F..FF...F                                                                         